#Belajar Pyspark - Ekspresi SQL di Dataframe dengan `expr()`

Spark menyediakan berbagai API yang dapat dimanfaatkan sesuai kebutuhan. Ada lebih dari satu cara untuk dapat melakukan operasi tertentu.

Selain menggunakan fungsi-fungsi python, kita dapat menggunakan ekspresi SQL untuk memanipulasi sebuah DataFrame. Ekspresi SQL tersebut dieksekusi menggunakan fungsi expr().  Pada artikel ini, kita akan membahas fungsi tersebut beserta contoh penggunaannya.


In [25]:
%pip install pyspark

In [26]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [27]:
spark = SparkSession.builder.appName("Belajar PySpark - expr").getOrCreate()

Sebelumnya kita buat DataFrame dari sebuah python list, lalu kita akan lakukan transformasi terhadap DataFrame tersebut.


In [28]:
data = [['Agus','Fisika',100],['Windy','Fisika',200],
        ['Budi','Biologi',200],['Dina','Fisika',150],
        ['Bayu','Fisika',50],['Dedi','Biologi',50]]

kolom = ["nama","jurusan","nilai"]

df = spark.createDataFrame(data,kolom)

df.show()

+-----+-------+-----+
| nama|jurusan|nilai|
+-----+-------+-----+
| Agus| Fisika|  100|
|Windy| Fisika|  200|
| Budi|Biologi|  200|
| Dina| Fisika|  150|
| Bayu| Fisika|   50|
| Dedi|Biologi|   50|
+-----+-------+-----+



Dengan fungsi expr() kita dapat menggunakan fungsi-fungsi SQL yang sudah familiar untuk kita. Ekspresi SQL tersebut dapat digunakan untuk melakukan update, filter, agregasi, maupun menambah kolom baru.


##Mengubah nilai kolom menjadi huruf kapital

Untuk mengkonversi nilai sebuah kolom menjadi huruf besar, dapat menggunakan fungsi SQL upper() dan fungsi withColumn()


In [29]:
df.withColumn("nama", F.expr("upper(nama)")).show()

+-----+-------+-----+
| nama|jurusan|nilai|
+-----+-------+-----+
| AGUS| Fisika|  100|
|WINDY| Fisika|  200|
| BUDI|Biologi|  200|
| DINA| Fisika|  150|
| BAYU| Fisika|   50|
| DEDI|Biologi|   50|
+-----+-------+-----+



##Memfilter DataFrame

Filter DataFrame menggunakan ekspresi SQL


In [30]:
df.filter(F.expr("nilai > 150")).show()

+-----+-------+-----+
| nama|jurusan|nilai|
+-----+-------+-----+
|Windy| Fisika|  200|
| Budi|Biologi|  200|
+-----+-------+-----+



Filter menggunakan gabungan beberapa kondisi


In [31]:
df.filter(F.expr("nama LIKE '%in%' AND nilai > 150")).show()

+-----+-------+-----+
| nama|jurusan|nilai|
+-----+-------+-----+
|Windy| Fisika|  200|
+-----+-------+-----+



##Agregasi

Agregasi DataFrame menggunakan ekspresi SQL


In [32]:
df.groupBy("jurusan").agg(F.expr("avg(nilai) as nilai_rata2")).show()

+-------+-----------+
|jurusan|nilai_rata2|
+-------+-----------+
| Fisika|      125.0|
|Biologi|      125.0|
+-------+-----------+



##Menambah kolom baru dari kolom yang sudah ada

Untuk memilih atau menambah kolom baru dari sebuah DataFrame, kita dapat gunakan perintah `DataFrame.select()`

In [33]:
df.select(F.col("*"), F.expr("upper(nama) as nama1")).show()

+-----+-------+-----+-----+
| nama|jurusan|nilai|nama1|
+-----+-------+-----+-----+
| Agus| Fisika|  100| AGUS|
|Windy| Fisika|  200|WINDY|
| Budi|Biologi|  200| BUDI|
| Dina| Fisika|  150| DINA|
| Bayu| Fisika|   50| BAYU|
| Dedi|Biologi|   50| DEDI|
+-----+-------+-----+-----+



Perintah di atas dapat ditulis dengan lebih singkat dengan menggunakan perintah `selectExpr()` yang merupakan fungsi DataFrame.

In [34]:
df.selectExpr("*", "upper(nama) as nama1").show()

+-----+-------+-----+-----+
| nama|jurusan|nilai|nama1|
+-----+-------+-----+-----+
| Agus| Fisika|  100| AGUS|
|Windy| Fisika|  200|WINDY|
| Budi|Biologi|  200| BUDI|
| Dina| Fisika|  150| DINA|
| Bayu| Fisika|   50| BAYU|
| Dedi|Biologi|   50| DEDI|
+-----+-------+-----+-----+



Untuk mengekstraksi kolom sebuah DataFrame menggunakan ekspresi SQL, sebaiknya memilih selectExpr() dibandingkan expr(). Disamping sintaksnya lebih singkat dan jelas, kita juga tidak perlu melakukan import spark.sql.functions.


##Memilih beberapa kolom dengan ekspresi SQL
Fungsi selectExpr() dapat menerima beberapa ekspresi SQL sekaligus. Oleh karena itu fungsi ini dapat mengembalikan beberapa kolom dari beberapa ekspresi SQL.


In [37]:
df.selectExpr("upper(nama) as nama1",
              "upper(jurusan) as jurusan1",
              "nilai").show()

+-----+--------+-----+
|nama1|jurusan1|nilai|
+-----+--------+-----+
| AGUS|  FISIKA|  100|
|WINDY|  FISIKA|  200|
| BUDI| BIOLOGI|  200|
| DINA|  FISIKA|  150|
| BAYU|  FISIKA|   50|
| DEDI| BIOLOGI|   50|
+-----+--------+-----+



##Ekspresi SQL yang kompleks

Fungsi expr() juga dapat mengeksekusi ekspresi SQL yang kompleks, seperti misalnya statement kondisional menggunakan CASE WHEN.

In [13]:
df.withColumn("kode_jurusan",
  F.expr("CASE WHEN jurusan = 'Fisika' THEN 'F'"
    " WHEN jurusan = 'Biologi' THEN 'B'"
    " ELSE 'NA' END")).show()

+-----+-------+-----+------------+
| nama|jurusan|nilai|kode_jurusan|
+-----+-------+-----+------------+
| Agus| Fisika|  100|           F|
|Windy| Fisika|  200|           F|
| Budi|Biologi|  200|           B|
| Dina| Fisika|  150|           F|
| Bayu| Fisika|   50|           F|
| Dedi|Biologi|   50|           B|
+-----+-------+-----+------------+



Untuk mengeksekusi beberapa ekspresi SQL sekaligus, misalnya untuk memilih beberapa kolom, gunakan fungsi selectExpr()

In [39]:
df.selectExpr("nama","jurusan",
  "(CASE WHEN jurusan = 'Fisika' THEN 'F'"
      " WHEN jurusan = 'Biologi' THEN 'B'"
      " ELSE 'NA' END) as kode_jurusan",
  "(CASE WHEN nilai < 100 THEN 'C'"
      " WHEN nilai < 200 THEN 'B'"
      " ELSE 'A' END) as kode_nilai",
  ).show()

+-----+-------+------------+----------+
| nama|jurusan|kode_jurusan|kode_nilai|
+-----+-------+------------+----------+
| Agus| Fisika|           F|         B|
|Windy| Fisika|           F|         A|
| Budi|Biologi|           B|         A|
| Dina| Fisika|           F|         B|
| Bayu| Fisika|           F|         C|
| Dedi|Biologi|           B|         C|
+-----+-------+------------+----------+

